# अध्याय 8 चैटबॉट

<div class='toc'>
<उल क्लास='टोक-आइटम'>
<li><span><a href='#一 परिचय' data-toc-modified-id='1. परिचय'>1. परिचय</a></span></li>
<ली>
<span><a href='#二性的与 बैकग्राउंड-बिल्डिंग ऑफ अंडरस्टैंडिंग' डेटा-टोक-संशोधित-id='2. बिल्ट-इन आइडेंटिटी'>2. बिल्ट-इन आइडेंटिटी</a></span> </li> <li>
<span><a href='#三 ऑर्डरिंग रोबोट' data-toc-modified-id='3. ऑर्डरिंग रोबोट'>3. ऑर्डरिंग रोबोट</a></span>
<उल क्लास='टोक-आइटम'>
<li><span><a href='#31-JSON सारांश बनाएं' data-toc-modified-id='3.1 json सारांश बनाएं'>3.1 json सारांश बनाएं</a></span></li>
</ul>
</li>
</ul>
</div>

## 1 परिचय

एक बड़े भाषा मॉडल का उपयोग करने के बारे में रोमांचक चीजों में से एक यह है कि हम इसका उपयोग बहुत कम प्रयास के साथ एक अनुकूलित चैटबॉट बनाने के लिए कर सकते हैं, इस अनुभाग में, हम यह पता लगाएंगे कि वैयक्तिकृत (या किसी के लिए विशेष) के साथ विस्तारित बातचीत के लिए चैट का लाभ कैसे उठाया जाए विशिष्ट कार्य या व्यवहार) चैटबॉट।

In [1]:
import openai
# तृतीय-पक्ष लाइब्रेरी आयात करें

openai.api_key = "sk-..."
# API_KEY सेट करें, कृपया इसे अपनी API_KEY से बदलें

चैटजीपीटी जैसे चैट मॉडल वास्तव में संदेशों की एक श्रृंखला को इनपुट के रूप में लेने और आउटपुट के रूप में एक मॉडल-जनित संदेश को वापस करने के लिए इकट्ठे किए जाते हैं। यह चैट प्रारूप मूल रूप से मल्टी-टर्न वार्तालापों को सुविधाजनक बनाने के लिए डिज़ाइन किया गया था, लेकिन हम पिछले अध्ययनों से जानते हैं कि यह भी उपयोगी है **एकल-मोड़ वाले कार्य** जिनमें कोई बातचीत शामिल नहीं है।

## 2. पहचान और संदर्भ निर्माण

आगे, हम दो सहायक कार्यों को परिभाषित करेंगे।

पहली विधि पूरे ट्यूटोरियल के लिए आपके पास रही है, ```get_completion```, और यह एक ही मोड़ के लिए काम करती है। हम प्रॉम्प्ट को किसी प्रकार के उपयोगकर्ता संदेश संवाद में डालते हैं। दूसरे को ```get_completion_from_messages`` कहा जाता है `, और इसके लिए संदेशों की एक सूची की आवश्यकता होती है। ये संदेश कई अलग-अलग भूमिकाओं से आ सकते हैं, जिनका हम वर्णन करेंगे।

पहले संदेश में, हम सिस्टम के रूप में एक सिस्टम संदेश भेजते हैं, जो एक समग्र सिस्टम संदेश प्रदान करता हैसहायक के व्यवहार और भूमिका को स्थापित करने में मदद करें, और बातचीत के लिए उच्च-स्तरीय संकेतों के रूप में कार्य करें। आप इसे सहायक के कान में फुसफुसाहट, उसकी प्रतिक्रिया का मार्गदर्शन करने के रूप में सोच सकते हैं, और उपयोगकर्ता सिस्टम संदेश पर ध्यान नहीं देगा उपयोगकर्ता, यदि आपने कभी चैटजीपीटी का उपयोग किया है, तो आप कभी नहीं जान पाएंगे कि चैटजीपीटी के सिस्टम संदेश क्या थे, और यह जानबूझकर है सिस्टम संदेशों का लाभ यह है कि वे डेवलपर्स को सहायक का मार्गदर्शन करने और उसकी प्रतिक्रिया को निर्देशित करने का एक तरीका प्रदान करते हैंस्वयं बातचीत का हिस्सा अनुरोध करें.

चैटजीपीटी वेब इंटरफ़ेस में, आपके संदेशों को उपयोगकर्ता संदेश कहा जाता है, और चैटजीपीटी के संदेशों को सहायक संदेश कहा जाता है, लेकिन चैटबॉट बनाते समय, सिस्टम संदेश भेजने के बाद, आपकी भूमिका का उपयोग केवल उपयोगकर्ता के रूप में किया जा सकता है; उपयोगकर्ता और सहायक, वार्तालाप संदर्भ प्रदान करते हैं।

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 控制模型输出的随机程度
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
# प्रिंट(str(response.choices[0].message))
    return response.choices[0].message["content"]

आइए अब बातचीत में इन संदेशों का उपयोग करने का प्रयास करें। हम इन संदेशों से प्रतिक्रिया प्राप्त करने के लिए उपरोक्त फ़ंक्शन का उपयोग करेंगे, और साथ ही, उच्च तापमान का उपयोग करेंगे (उच्च तापमान अधिक विविधता उत्पन्न करते हैं, इस पर अध्याय 7 में अधिक जानकारी दी गई है)।

सिस्टम संदेश कहता है, आप एक सहायक हैं जो शेक्सपियर की तरह बोलता है। इस प्रकार हम सहायक को वर्णन करते हैं **उसे कैसा व्यवहार करना चाहिए** फिर, पहला उपयोगकर्ता संदेश है *मुझे एक चुटकुला बताओ* सहायक है *ची ने ऐसा क्यों कियाक्या आप सड़क पार कर रहे हैं? * भेजा गया अंतिम उपयोगकर्ता संदेश है *मुझे नहीं पता*।

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, fair sir.


In [8]:
#中文
messages =  [  
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
{'role':'user', 'content':'给我讲个笑话'},   
{'role':'assistant', 'content':'鸡为什么过马路'},   
{'role':'user', 'content':'我不知道'}  ]

In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

因为它要去找“母鸡”。哈哈哈！（注：此为英文双关语，"chicken"是鸡的意思，也是胆小的意思；"cross the road"是过马路的意思，也是“破坏规则”的意思。）


(नोट: उपरोक्त उदाहरण में, तापमान = 1 के चयन के कारण, मॉडल के उत्तर अधिक यादृच्छिक और भिन्न होंगे (उनमें से कुछ बहुत रचनात्मक हैं)। यहां एक और उत्तर है:

मैं आपके प्रश्न का उत्तर शेक्सपियर की एक कविता से देता हूँ:

जब मुर्गे का दिल आगे बढ़ना चाहता है,
सड़क उसकी पसंद है.
धीरे-धीरे गाड़ी चलाओ और आसमान साफ़ है,
हार्न बजता है और आड़े-तिरछे वार होते हैं।

इससे पूछो कि कहाँ जाना है?
क्योंकि सड़क पर कोई संकेत नहीं है,
लेकिन मुर्गी आगे छलांग लगाती है,
और इसका निर्णय नहीं होता हैसंकोच करने की जरूरत है.

मुर्गे की बुद्धि अवर्णनीय है,
सड़क काली लाह की तरह सुनसान है.
लेकिन इसका साहस सराहनीय है,
बिना पीछे हटे बहादुरी से आगे बढ़ना।

तो मुर्गी सड़क क्यों पार करती है?
यह शोरगुल वाले यातायात के संकट को सहन करता है।
अपनी दहाड़ के कारण वह साहसपूर्वक छलांग लगाता है,
और एक घमंडपूर्ण भित्तिचित्र बनाता है।

तो मजाक की खूबसूरती,
साथ ही मुर्गे का साहस उमड़ पड़ा।
सड़क के डर के बिना जीवन के बारे में हँसना,
बुद्धि और साहस के साथ, यह अपनी सुंदरता दिखाता है।

मुझे उम्मीद हैशेक्सपियर का यह उत्तर आपको कुछ खुशी देगा!

आइए एक और उदाहरण देखें। सहायक का संदेश है *आप एक मिलनसार चैटबॉट हैं* और पहला उपयोगकर्ता संदेश है *हाय, मेरा नाम ईसा* है। हम पहला उपयोगकर्ता संदेश प्राप्त करना चाहते हैं।

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's great to meet you. How can I assist you today?


In [11]:
#中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'Hi, 我是Isa。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

嗨，Isa，很高兴见到你！有什么我可以帮助你的吗？


आइए एक और उदाहरण आज़माएँ। सिस्टम संदेश है, आप एक मिलनसार चैटबॉट हैं, और पहला उपयोगकर्ता संदेश है, हाँ, क्या आप मुझे याद दिला सकते हैं कि मेरा नाम क्या है?

In [12]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but since we don't have any personal information about you, I don't know your name. Can you please tell me your name?


In [14]:
#中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

抱歉，我不知道您的名字，因为我们是虚拟的聊天机器人和现实生活中的人类在不同的世界中。


जैसा कि आप ऊपर देख सकते हैं, मॉडल वास्तव में मेरा नाम नहीं जानती।

इसलिए, भाषा मॉडल के साथ प्रत्येक इंटरैक्शन दूसरों से स्वतंत्र है, जिसका अर्थ है कि यदि हम चाहते हैं कि मॉडल वार्तालाप के पहले के हिस्सों को संदर्भित करे या "याद रखे" तो हमें मॉडल को वर्तमान वार्तालाप में संदर्भित करने के लिए सभी प्रासंगिक संदेश प्रदान करने होंगे , हमें मॉडल के इनपुट में पहले के एक्सचेंज प्रदान करने होंगे। हम इस संदर्भ को निम्नलिखित उदाहरण कहते हैं।

In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


यहाँ एक और उत्तर है:

*तुम्हारा नाम ईसा है! मैं कैसे भूल सकता हूँ?*

In [16]:
#中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然可以！您的名字是Isa。


अब जब हमने मॉडल संदर्भ दे दिया है, जो कि पिछली बातचीत में मेरा नाम बताया गया है, तो हम वही प्रश्न पूछ सकते हैं, जो कि मेरा नाम क्या है क्योंकि मॉडल में सभी आवश्यक संदर्भ हैं, यह इस प्रकार प्रतिक्रिया देने में सक्षम है हमने दर्ज किए गए संदेशों की सूची देखी।

## 3. रोबोट ऑर्डर करना

अब, हम एक "ऑर्डरिंग रोबोट" बनाते हैं जो स्वचालित रूप से उपयोगकर्ता की जानकारी एकत्र करता है और पिज्जा की दुकानों से ऑर्डर स्वीकार करता है।

निम्नलिखित फ़ंक्शन हमारे उपयोगकर्ता संदेशों को एकत्र करेगा ताकि हम मैन्युअल इनपुट से बच सकें जैसा कि हमने अभी किया था। यह फ़ंक्शन हमारे द्वारा नीचे बनाए गए उपयोगकर्ता इंटरफ़ेस से प्रॉम्प्ट एकत्र करेगा, फिर इसे संदर्भ (``संदर्भ```) नामक सूची में जोड़ देगा। ), और हर बार मॉडल को कॉल करने पर उस संदर्भ का उपयोग करें, मॉडल की प्रतिक्रिया भी संदर्भ में जोड़ी जाती है, इसलिए उपयोगकर्ता दोनोंसंदेश और मॉडल संदेश को संदर्भ में जोड़ा जाता है, और संदर्भ धीरे-धीरे लंबा हो जाता है, इस तरह, मॉडल के पास यह निर्धारित करने के लिए आवश्यक जानकारी होती है कि आगे क्या करना है।

In [19]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

अब हम ऑर्डर बॉट को प्रदर्शित करने के लिए इस यूआई को सेट अप और चलाएंगे। प्रारंभिक संदर्भ में मेनू वाला सिस्टम संदेश होता है और प्रत्येक आमंत्रण पर इसका उपयोग किया जाता है। जैसे-जैसे बातचीत आगे बढ़ेगी।

In [ ]:
!pip install panel

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ] # संदेश संचित करें


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

चल रहे परिणाम इंटरैक्टिव हैं, कृपया नीचे चीनी संस्करण देखें।

### 3.1 JSON डाइजेस्ट बनाना

यहां हम मॉडल से JSON सारांश बनाने के लिए भी कहते हैं जिसे हम ऑर्डरिंग सिस्टम को भेज सकते हैं।

इसलिए हमें एक अन्य निर्देश के रूप में संदर्भ में एक और सिस्टम संदेश जोड़ने की आवश्यकता है। हम कहते हैं *अभी ऑर्डर किए गए ऑर्डर का एक JSON सारांश बनाएं, जिसमें 1) पिज़्ज़ा, आकार सहित, 2) टॉपिंग की सूची के साथ प्रत्येक आइटम की कीमत सूचीबद्ध हो। , 3) पेय की सूची, 4) आकार सहित पक्षों की सूची, और अंत में कुल कीमत* इसे एक उपयोगकर्ता संदेश के रूप में भी परिभाषित किया जा सकता है, जरूरी नहीं कि यह एक सिस्टम संदेश हो।

ध्यान देंहम यहां कम तापमान का उपयोग करते हैं क्योंकि इस प्रकार के कार्यों के लिए, हम चाहते हैं कि आउटपुट अपेक्षाकृत पूर्वानुमानित हो।

In [45]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "cheese",
    "size": "large",
    "toppings": [
      "mushrooms"
    ],
    "price": 12.45
  },
  "drinks": [
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    },
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 18.45
}
``` 

Note: I assumed that the price of the large cheese pizza with mushrooms is $12.45 instead of $12.95, since the customer only ordered one topping.


In [76]:
#中文
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ] # संदेश संचित करें


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [77]:
dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [78]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

以下是上一个食品订单的 JSON 摘要：

```
{
  "order": {
    "pizza": {
      "type": "芝士披萨",
      "size": "大",
      "price": 10.95
    },
    "toppings": [
      {
        "name": "蘑菇",
        "price": 1.5
      }
    ],
    "drinks": [
      {
        "name": "雪碧",
        "size": "大",
        "price": 3
      },
      {
        "name": "雪碧",
        "size": "大",
        "price": 3
      }
    ],
    "sides": [],
    "total_price": 18.45
  }
}
```


अब जब हमने अपना खुद का खाना ऑर्डर करने वाला चैटबॉट बना लिया है, तो चैटबॉट के व्यवहार को बदलने और इसे अलग-अलग भूमिकाएं निभाने और अलग-अलग ज्ञान देने के लिए सिस्टम संदेशों को अनुकूलित और संशोधित करने के लिए स्वतंत्र महसूस करें।

परिशिष्ट: निम्नलिखित चित्र ऑर्डर करने वाले रोबोट की पूरी बातचीत प्रक्रिया को दर्शाता है:
![image.png](../../figures/Chatbot-पिज्जा-cn.png)